In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from math import sqrt
from sklearn.metrics import mean_squared_error
from statsmodels.iolib.summary2 import summary_col
import itertools
import matplotlib.pyplot as plt

In [2]:
# Read only total_tract_population column from acs_data.csv
tract_pop = pd.read_csv(
    "../data/acs_data_race_2022.csv",
    usecols=["tract", "total_tract_population"],
    dtype={"total_tract_population": np.float32, "tract": str},
)
tract_pop.columns

Index(['tract', 'total_tract_population'], dtype='object')

# Run Regression 

In [3]:
# Load the data
dtype_dict = {
    "B01003_001E_adj_supply_any_avg_tract": np.float32,
    "B01003_001E_adj_supply_hr_avg_tract": np.float32,
    "B01003_001E_totcost_any_avg_tract": np.float32,
    "state": str,
    "county": str,
    "tract": str,
    "median_household_income": np.float32,
    "gini_index": np.float32,
    "educational_attainment": np.float32,
    "employment_status": np.float32,
    "housing_tenure": np.float32,
    "poverty_status": np.float32,
    "commute_time": np.float32,
    "without_health_insurance": np.float32,
    "white": np.float32,
    "black": np.float32,
    "am_indian_alaska_native": np.float32,
    "asian": np.float32,
    "other_race": np.float32,
    "hispanic_latino": np.float32,
    "density": np.float32,
    "white_share_state": np.float32,
    "black_share_state": np.float32,
    "am_indian_alaska_native_share_state": np.float32,
    "asian_share_state": np.float32,
    "other_race_share_state": np.float32,
    "hispanic_latino_share_state": np.float32,
}


data = pd.read_csv(
    "../data/acs_data_all_2022.csv", dtype=dtype_dict
)  # Replace with the path to your data file

# Merge tract_pop to data
data = data.merge(tract_pop, how="left", on="tract", validate="1:1")

# Scale median_household_income by 10,000
data["median_household_income"] = data["median_household_income"] / 10000

# Scale density by 10,000
data["density"] = data["density"] / 10000

# Rename the variables
data.rename(
    columns={
        "B01003_001E_adj_supply_any_avg_tract": "Average Supply of Any Provider",
        "B01003_001E_adj_supply_hr_avg_tract": "Average Supply of HR Provider",
        "B01003_001E_totcost_any_avg_tract": "Average Total Cost of Any Provider",
        "median_household_income": "Median Household Income",
        "gini_index": "Gini Index",
        "educational_attainment": "Educational Attainment",
        "employment_status": "Share Employed",
        "housing_tenure": "Housing Tenure",
        "poverty_status": "Share in Poverty",
        "commute_time": "Commute Time",
        "without_health_insurance": "Without Health Insurance",
        "white": "Share of White Population",
        "black": "Share of Black Population",
        "am_indian_alaska_native": "Share of American Indian/Alaska Native Population",
        "asian": "Share of Asian Population",
        "other_race": "Share of Other Race Population",
        "hispanic_latino": "Share of Hispanic/Latino Population",
        "density": "Population Density",
        "tract_pop": "total_tract_population",
        "white_share_state": "Share of White Population State",
        "black_share_state": "Share of Black Population State",
        "am_indian_alaska_native_share_state": "Share of American Indian/Alaska Native Population State",
        "asian_share_state": "Share of Asian Population State",
        "other_race_share_state": "Share of Other Race Population State",
        "hispanic_latino_share_state": "Share of Hispanic/Latino Population State",
    },
    inplace=True,
)

# calculate state population
state_population = data.groupby("state")["total_tract_population"].transform("sum")

# create tract_weight
data["tract_weight"] = data["total_tract_population"] / state_population

In [4]:
data[["tract","Share of White Population", "Share of Asian Population"]].head()

,tract,Share of White Population,Share of Asian Population
0,27001770100,0.911019,0.002079
1,27001770200,0.917112,0.000446
2,27001770300,0.955981,0.000000
3,27001770401,0.927431,0.000000
4,27001770402,0.847438,0.006333


In [5]:
data.head()

,Average Supply of Any Provider,Average Supply of HR Provider,Average Total Cost of Any Provider,state,county,tract,Median Household Income,Gini Index,Educational Attainment,Share Employed,...,Share of Hispanic/Latino Population,Population Density,Share of White Population State,Share of Black Population State,Share of American Indian/Alaska Native Population State,Share of Asian Population State,Share of Other Race Population State,Share of Hispanic/Latino Population State,total_tract_population,tract_weight
0,0.048920,0.022648,143.333374,27,27001,27001770100,5.4737,0.4147,0.098657,0.454120,...,0.010811,0.000234,0.000493,0.000019,0.000712,0.000018,0.000689,0.000081,2405.0,0.000426
1,0.355802,0.000000,106.816559,27,27001,27001770200,5.3304,0.3752,0.093564,0.435527,...,0.000891,0.000222,0.000463,0.000005,0.001510,0.000004,0.000535,0.000006,2244.0,0.000397
2,0.748800,0.281424,145.680359,27,27001,27001770300,4.5583,0.4545,0.175796,0.486154,...,0.008804,0.004220,0.000709,0.000040,0.000625,0.000000,0.000347,0.000091,3294.0,0.000583
3,0.121860,0.000000,132.144562,27,27001,27001770401,5.8125,0.4224,0.138640,0.455004,...,0.037010,0.000628,0.000288,0.000013,0.000625,0.000000,0.000072,0.000159,1378.0,0.000244
4,0.815553,0.000000,41.656166,27,27001,27001770402,4.7500,0.4169,0.100486,0.463016,...,0.019574,0.000220,0.000331,0.000022,0.002609,0.000039,0.000439,0.000106,1737.0,0.000307


In [6]:
def run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=1,
):
    regression_results = {}
    rmse_results = {}

    for covariates, model_name in zip(covariates_list, model_names):
        X = sm.add_constant(data[covariates])
        y = data[outcome_variable] * scalar

        # Check if weights_variable is provided
        if weights_variable:
            # Create weights array
            weights = data[weights_variable]
            model = sm.WLS(y, X, weights=weights)
        else:
            model = sm.OLS(y, X)

        result = model.fit()

        rmse = np.sqrt(mean_squared_error(y, result.predict()))

        # Use model name as a key to store results
        regression_results[model_name] = result
        rmse_results[model_name] = rmse

    return regression_results, rmse_results

### Access to Any Provider

In [7]:
# Define the covariates

# First set of covariates will include racial group shares
first_set = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
]

# Second set of covariates will include the first set plus educational attainment, median income
second_set = first_set + [
    "Median Household Income",
    "Educational Attainment",
]

# Third set of covariates will include the second set plus age, population density, and unemployment rate
third_set = second_set + [
    "Gini Index",
    "Share Employed",
    "Housing Tenure",
    "Share in Poverty",
    "Commute Time",
    "Without Health Insurance",
]

# The fourth set of covariates will include the third set plus the population density
fourth_set = third_set + ["Population Density"]

# Combine all covariates into a list of lists
covariates_list = [first_set, second_set, third_set, fourth_set]

# Define the outcome variable
outcome_variable = "Average Supply of Any Provider"

# Define the scalar
scalar = 1  # Replace with your desired scalar

# Define model names
model_names = ["First Set", "Second Set", "Third Set", "Fourth Set"]

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [8]:
# Variables are in order
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Share Employed",
    "Housing Tenure",
    "Share in Poverty",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,0.098,-0.034,-0.153*,-0.191*
,(0.083),(0.084),(0.092),(0.099)
Share of American Indian/Alaska Native Population,-0.045,0.070,0.256,0.253
,(0.195),(0.193),(0.209),(0.209)
Share of Asian Population,-0.630***,-0.608***,-0.639***,-0.650***
,(0.113),(0.111),(0.112),(0.112)
Share of Other Race Population,0.494*,0.038,-0.126,-0.141
,(0.293),(0.288),(0.286),(0.287)
Share of Hispanic/Latino Population,0.225*,0.177,0.076,0.052
,(0.130),(0.129),(0.133),(0.135)


In [9]:
summary_df_ordered.to_csv("../data/any_regression_2022.csv")

### High Rated Providers

In [10]:
# Define the outcome variable
outcome_variable = "Average Supply of HR Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [11]:
# Variables are in order of the regression table
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Share Employed",
    "Housing Tenure",
    "Share in Poverty",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,0.271***,0.218***,0.144***,0.037
,(0.045),(0.045),(0.049),(0.052)
Share of American Indian/Alaska Native Population,0.015,0.127,0.169,0.161
,(0.105),(0.103),(0.111),(0.110)
Share of Asian Population,-0.080,-0.088,-0.123**,-0.155***
,(0.061),(0.059),(0.059),(0.059)
Share of Other Race Population,0.453***,0.171,0.111,0.068
,(0.158),(0.154),(0.152),(0.151)
Share of Hispanic/Latino Population,0.212***,0.218***,0.173**,0.103
,(0.070),(0.069),(0.071),(0.071)


In [12]:
summary_df_ordered.to_csv("../data/hr_regression_2022.csv")

### Total Cost

In [13]:
# Define the outcome variable
outcome_variable = "Average Total Cost of Any Provider"

# Call the function to run the regression analysis
regression_results, rmse_results = run_regression_analysis(
    covariates_list,
    outcome_variable,
    data,
    model_names,
    weights_variable="total_tract_population",
    scalar=scalar,
)

# Generate the summary table
summary_table = summary_col(
    list(regression_results.values()),
    stars=True,
    float_format="%.3f",
    model_names=model_names,
    info_dict={
        "N": lambda x: f"{x.nobs:.0f}",
        "R2": lambda x: f"{x.rsquared:.3f}",
        "Adjusted R2": lambda x: f"{x.rsquared_adj:.3f}",
    },
)

# Convert the summary table to a DataFrame
summary_df1 = pd.DataFrame(summary_table.tables[0])

# Convert RMSE results to a DataFrame
rmse_df = pd.DataFrame(rmse_results, index=["RMSE"])

# Append the RMSE results to the summary DataFrame
summary_df1 = pd.concat([summary_df1, rmse_df])

In [14]:
# Variables are in order of the regression table
covariate_list_ordered = [
    "Share of Black Population",
    "Share of American Indian/Alaska Native Population",
    "Share of Asian Population",
    "Share of Other Race Population",
    "Share of Hispanic/Latino Population",
    "Median Household Income",
    "Educational Attainment",
    "Gini Index",
    "Share Employed",
    "Housing Tenure",
    "Share in Poverty",
    "Commute Time",
    "Without Health Insurance",
    "Population Density",
    "const",
]

# Separate list for non-standard error metrics
metrics_list = ["N", "R2", "Adjusted R2", "RMSE"]

# Last non-empty row name
last_non_empty = ""

# New index
new_index = []

for name in summary_df1.index:
    if name == "":
        new_index.append(last_non_empty + "_se")
    else:
        new_index.append(name)
        last_non_empty = name

# Assign the new index to summary_df1
summary_df1.index = new_index

# Create a new list that alternates between each variable name and its corresponding "_se" version
covariate_list_ordered_extended = [
    item
    for sublist in [[var, var + "_se"] for var in covariate_list_ordered]
    for item in sublist
    if item in summary_df1.index
]

# Add the non-standard error metrics to the list
covariate_list_ordered_extended += metrics_list

# Reindex summary_df1 according to covariate_list_ordered_extended
summary_df_ordered = summary_df1.reindex(covariate_list_ordered_extended)

# Specify the indexes that end with '_se' that you want to replace
indexes_to_replace = [name for name in summary_df_ordered.index if name.endswith("_se")]

# Create a mapping dictionary to replace specified indexes with an empty string
index_replace_dict = {index: "" for index in indexes_to_replace}

# Use the .rename method with the mapping dictionary
summary_df_ordered.rename(index=index_replace_dict, inplace=True)

summary_df_ordered

,First Set,Second Set,Third Set,Fourth Set
Share of Black Population,122.581***,203.455***,230.452***,211.165***
,(17.124),(13.148),(13.992),(14.903)
Share of American Indian/Alaska Native Population,-256.077***,-57.460*,33.480,31.983
,(40.199),(30.065),(31.750),(31.621)
Share of Asian Population,180.861***,94.893***,109.695***,104.000***
,(23.334),(17.362),(16.978),(16.979)
Share of Other Race Population,354.841***,226.629***,198.088***,190.476***
,(60.363),(44.969),(43.478),(43.348)
Share of Hispanic/Latino Population,-21.025,112.558***,124.092***,111.443***
,(26.701),(20.094),(20.258),(20.469)


In [15]:
summary_df_ordered.to_csv("../data/cost_regression_2022.csv")

In [16]:
# Create a scatter plot of share of black population vs access to any provider
# scatter_plot(
# x, y, x_label="Variable 1", y_label="Variable 2", title="Example Scatter Plot"
# )